## Bibliotecas

In [25]:
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Estimativa da memória

- O ambiente utilizado para essa execução é um WSL com Ubunto como OS
- No momento da execução, há MemFree é a quantidade de memória livre, do total (MemTotal) do sistema

O Valor da memória livre varia dependendo dos programas em execução

In [17]:
with open("/proc/meminfo", 'r') as memory_status:
    lines = memory_status.readlines()
    memtotal = int(lines[0].split('kB')[0].split(':')[1])
    print(lines[0])
    print(lines[1])
    

MemTotal:       16469120 kB

MemFree:        12003612 kB



O consumo de memória dos programa se dá, manjoritariamente, pela alocação da matriz, ou seja, complexidade de espaço $O(n^2)$. Dessa forma, assumindo que os programas podem utilizar toda a memória do sistema, temos que:

$$
    n_{max}^2  \cdot size(e) \leq mtotal \\
   
$$
$$
 n_{max} \leq \sqrt{\frac{mtotal}{size(e)}}
$$

Sendo $n_{max}$ o tamanho máximo da dimensão da matriz, $size(e)$ o número de bytes necessário para armazenar um elemento da matriz e  $mfree$ a quantidade de memória livre em bytes.


Nesse sentindo, supondo que a memória impressa anteriormente estará livre e disponível para a alocação pelo programador, no momento da execução do programa, teríamos os seguintes valores de $n_{max}$ para Fortran utilizando tipo $Real$ ( $size(e) = 16$) e para C utilizando tipo $float$ ($size(e) = 4$).


In [18]:
n_max_fortran =  round(np.sqrt(memtotal*1000/16))
n_max_c = round(np.sqrt(memtotal*1000/4))
n_max_fortran, n_max_c

(32083, 64166)

## Estimativa de Tempo para cada N

### Para o programa em Fortran
<hr/>

- Sem inverter o loop

In [14]:
csv_file = 'results/fortran_lc.csv'
with open(csv_file, 'a') as f:
    f.write('n,tempo\n')

In [15]:
n = 2
while True:
    try:
        out = subprocess.check_output(["./src/program1.out", f'{n}', '0'])
    except subprocess.CalledProcessError as e:
        print(e.output)
        break
    else: 
        with open(csv_file, 'a') as f:
            f.write(f'{n},{float(out)}\n')
        n *= 2

b''


Operating system error: Cannot allocate memory
Allocation would exceed memory limit

Error termination. Backtrace:
#0  0x7f97a53a0d5a
#1  0x7f97a53a1869
#2  0x7f97a53a1ac4
#3  0x7f97a568f518
#4  0x7f97a568fbe4
#5  0x7f97a51a4082
#6  0x7f97a568f16d
#7  0xffffffffffffffff


- Invertendo o loop

In [57]:
csv_file = 'results/fortran_cl.csv'
with open(csv_file, 'a') as f:
    f.write('n, tempo\n')

In [58]:
n = 2
while True:
    try:
        out = subprocess.check_output(["./src/program1.out", f'{n}', '1'])
    except subprocess.CalledProcessError as e:
        print(e.output)
        break
    else: 
        with open(csv_file, 'a') as f:
            f.write(f'{n}, {float(out)}\n')
        n *= 2

b''


Operating system error: Cannot allocate memory
Allocation would exceed memory limit

Error termination. Backtrace:
#0  0x7fddd6620d5a
#1  0x7fddd6621869
#2  0x7fddd6621ac4
#3  0x7fddd6918518
#4  0x7fddd6918be4
#5  0x7fddd6424082
#6  0x7fddd691816d
#7  0xffffffffffffffff


### Para o programa em C
<hr/>

- Sem inverter o loop

In [59]:
csv_file = 'results/c_lc.csv'
with open(csv_file, 'a') as f:
    f.write('n,tempo\n')

In [60]:
n = 2
while True:
    try:
        out = subprocess.check_output(["./src/program2.out", f'{n}', '0'])
    except subprocess.CalledProcessError as e:
        print(e.output)
        break
    else: 
        with open(csv_file, 'a') as f:
            f.write(f'{n}, {float(out)}\n')
        n *= 2

b''


- Invertendo o loop

In [61]:
csv_file = 'results/c_cl.csv'
with open(csv_file, 'a') as f:
    f.write('n,tempo\n')

In [62]:
n = 2
while True:
    try:
        out = subprocess.check_output(["./src/program2.out", f'{n}', '1'])
    except subprocess.CalledProcessError as e:
        print(e.output)
        break
    else: 
        with open(csv_file, 'a') as f:
            f.write(f'{n}, {float(out)}\n')
        n *= 2

b''


## Análise